In [3]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
from Bio import AlignIO

In [4]:
namespace1="broad-genomics-delivery"
workspace1="Getz_IBM_CellLines_Exomes"
namespace2="broad-firecloud-ccle"
workspace2="CCLE_DepMap_WES"
namespace3="broad-genomics-delivery"
workspace3="CCLE_DepMap_WES"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
source1="ibm"
source2="ccle"
source3="ccle"

samplesetname = "19Q3"
release = samplesetname

In [15]:
wm1 = dm.WorkspaceManager(namespace1, workspace1)
wm2 = dm.WorkspaceManager(namespace2, workspace2)
wm3 = dm.WorkspaceManager(namespace3, workspace3)

refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [34]:
newsamples = createDatasetWithNewCellLines(refwm, samplesetname, wm3, source3)

uploading new samples
Successfully imported 1597 participants.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 2 records
Successfully imported 1822 samples.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 663210 records
creating a sample set
Sample set "19Q3add" (14 samples) successfully updated.


# Run the pipeline

In [36]:
CGAmutations = refwm.get_config("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019")
CGAmutations

{'deleted': False,
 'inputs': {'CGA_Production_Analysis_Workflow.CrossCheckLaneFingerprints_Task.HaplotypeDBForCrossCheck': 'workspace.HAP_DB_FOR_CC',
  'CGA_Production_Analysis_Workflow.gatk_acnv_only.one_thousand_genomes_common_variants_minor_allele_freq_five': 'workspace.COMMON_VARIANTS_1000_GENOMES',
  'CGA_Production_Analysis_Workflow.VEP_Task.GNOMAD_FILE_IDX': 'workspace.GNOMAD_IDX',
  'CGA_Production_Analysis_Workflow.lego_plotter_task.mut_categs': 'workspace.MUT_CATEGORIES',
  'CGA_Production_Analysis_Workflow.MutectFC_Task.mutectIntervals': 'workspace.MUTECT_FC_INTERVALS',
  'CGA_Production_Analysis_Workflow.hasPicardMetrics_normal': '',
  'CGA_Production_Analysis_Workflow.DB_SNP_VCF_IDX': 'workspace.DB_SNP_VCF_IDX',
  'CGA_Production_Analysis_Workflow.iceMAFPonFilter.PONFile': 'workspace.ICE_TOKEN_PON',
  'CGA_Production_Analysis_Workflow.ctrlName': '"CCLF_RCRF1009-Normal-SM-F3R8L"',
  'CGA_Production_Analysis_Workflow.targetIntervals': 'workspace.ILLUMINA_TARGET_INTERVALS',


In [37]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

APIException: Failed to update existing configuration: (404) : {
  "statusCode": 404,
  "source": "rawls",
  "timestamp": 1562601418037,
  "causes": [],
  "stackTrace": [],
  "message": "Cannot get agora://cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debugging/22 from method repo."
}


Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/hound/client.py", line 60, in _bulk_upload
    assert proc.returncode == 0
AssertionError
/anaconda3/envs/py36/lib/python3.6/site-packages/hound/client.py:63: UserWarning: Fast record population failed. Switching to slower fallback
  warnings.warn("Fast record population failed. Switching to slower fallback")


In [ ]:
terra.waitForSubmission(refwm, submission_id)

In [4]:
filtercommonvar = refwm.get_config("common_variant_filter")
filtercommonvar

{'deleted': False,
 'inputs': {'CommonVariantFilter.maf': 'this.mutation_validator_validated_maf',
  'CommonVariantFilter.disable_whitelist': '',
  'CommonVariantFilter.docker_tag': '',
  'CommonVariantFilter.filter_noncoding': '',
  'CommonVariantFilter.min_filter_depth': '',
  'CommonVariantFilter.min_exac_ac': '',
  'CommonVariantFilter.preemptible': '',
  'CommonVariantFilter.sampleId': 'this.name',
  'CommonVariantFilter.SSD': '512',
  'CommonVariantFilter.RAM': '32'},
 'methodConfigVersion': 7,
 'methodRepoMethod': {'methodName': 'common_variant_filter',
  'methodVersion': 9,
  'methodNamespace': 'breardon',
  'methodUri': 'agora://breardon/common_variant_filter/9',
  'sourceRepo': 'agora'},
 'name': 'common_variant_filter',
 'namespace': 'breardon',
 'outputs': {'CommonVariantFilter.commonfilterTask.passCount': 'this.passCount_cga_cf',
  'CommonVariantFilter.commonfilterTask.passedMAF': 'this.passedMAF_cga_cf',
  'CommonVariantFilter.commonfilterTask.rejectCount': 'this.rejectCo

In [10]:
refwm.update_config(filtercommonvar)
submission_id = refwm.create_submission(filtercommonvar['name'], samplesetname, 'sample_set', expression='this.samples')

Successfully updated configuration breardon/common_variant_filter
Successfully created submission d220cdab-c7a6-4fad-9057-37254a9e5a6f.


In [44]:
terra.waitForSubmission(refwm, submission_id)

1.0 of jobs Succeeded in submission 0.sion 0. 10 mn elapsed.


[]

In [11]:
filtermaf = refwm.get_config("filterMAF_on_CGA_pipeline")
filtermaf

{'deleted': False,
 'inputs': {'filterMAF_for_CGA_workflow.filterMaf.inMAFfn': 'this.passedMAF_cga_cf',
  'filterMAF_for_CGA_workflow.filterMaf.onlyCoding': '"true"',
  'filterMAF_for_CGA_workflow.filterMaf.minAltReads': '"2"',
  'filterMAF_for_CGA_workflow.filterMaf.minAF': '"0.1"',
  'filterMAF_for_CGA_workflow.filterMaf.maxExAC_AF_COSMIC': '"1E-4"',
  'filterMAF_for_CGA_workflow.filterMaf.minCoverage': '"4"',
  'filterMAF_for_CGA_workflow.filterMaf.COSMIChotspotMinCnt': '"10"',
  'filterMAF_for_CGA_workflow.filterMaf.intersectWithMafFN': '',
  'filterMAF_for_CGA_workflow.filterMaf.addExAC_column': '"gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/scripts/add_column.R"',
  'filterMAF_for_CGA_workflow.filterMaf.sample_id': 'this.name',
  'filterMAF_for_CGA_workflow.filterMaf.TCGAhotspotMinCnt': '"3"',
  'filterMAF_for_CGA_workflow.filterMaf.TCGAhotspotMutFN': 'workspace.TCGAhotspotMutFN',
  'filterMAF_for_CGA_workflow.filterMaf.maxPon_loglike': '"1"',
  'filterMAF_for_CGA_workflow

In [12]:
refwm.update_config(filtermaf)
submission_id = refwm.create_submission(filtermaf['name'], samplesetname,'sample_set',expression='this.samples')

Successfully updated configuration gkugener/filterMAF_on_CGA_pipeline
Successfully created submission 2862638e-8eea-4d8d-8924-0f7fe868e278.


In [16]:
terra.waitForSubmission(refwm, submission_id)

ibm_ACH-001518
ibm_ACH-001616
ibm_ACH-001623
ibm_ACH-001687
ibm_ACH-001698
ibm_ACH-001719
ibm_ACH-001991
ibm_ACH-002019
ibm_ACH-002022
ibm_ACH-002023
ibm_ACH-002027
ibm_ACH-002042
ibm_ACH-002069
0.0 of jobs Succeeded in submission 0.


['ibm_ACH-001518',
 'ibm_ACH-001616',
 'ibm_ACH-001623',
 'ibm_ACH-001687',
 'ibm_ACH-001698',
 'ibm_ACH-001719',
 'ibm_ACH-001991',
 'ibm_ACH-002019',
 'ibm_ACH-002022',
 'ibm_ACH-002023',
 'ibm_ACH-002027',
 'ibm_ACH-002042',
 'ibm_ACH-002069']

In [48]:
# we use this function to merge 19Q3add in 19Q3
terra.updateAllSampleSet(refwm,'19Q3interim', Allsample_setname='19Q3')

Sample set "19Q3" (34 samples) successfully updated.


In [41]:
aggregate = refwm.get_config("aggregateMAFs_selectFields_copy")
aggregate

{'deleted': False,
 'inputs': {'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.outputFN_prfx': 'this.name',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.num_preempt': '5',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.disk_space': '1000',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.aggregate_selectedFields_MAF_Script': 'workspace.MAFscript',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.memory': '15',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs': 'this.samples.outMAFfn',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields': '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tumor_Sample_Barcode,cDNA_Change,Codon_Change,Protein_Change,t_alt_co

In [ ]:
refwm.update_config(aggregate)
submission_id = refwm.create_submission(aggregate['name'], samplesetname, sa)

### unfiltered

In [30]:
samplesetname = '19Q3'

In [78]:
aggregate['outputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.mergedMAFfn'] = 'this.unfiltered_CGA_MAF_aggregated'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs'] = 'this.samples.outMAFannotatedFN'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields'] = '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tumor_Sample_Barcode,cDNA_Change,Codon_Change,Protein_Change,t_alt_count,t_ref_count,tumor_f,Passed_Filters,failure_reasons"'
refwm.update_config(aggregate)
submission_id2 = refwm.create_submission(aggregate['name'], samplesetname)

Successfully updated configuration gkugener/aggregateMAFs_selectFields_copy
Successfully created submission a092a482-2ad2-4c86-9ecd-70e8f39907ae.


### filtered

In [79]:
aggregate['outputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.mergedMAFfn'] = 'this.filtered_CGA_MAF_aggregated'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs'] = 'this.samples.outMAFfn'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields'] = '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tumor_Sample_Barcode,cDNA_Change,Codon_Change,Protein_Change,t_alt_count,t_ref_count,tumor_f,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,i_ExAC_AF,PASS"'
refwm.update_config(aggregate)
submission_id1 = refwm.create_submission(aggregate['name'], samplesetname)

Successfully updated configuration gkugener/aggregateMAFs_selectFields_copy
Successfully created submission 275c53c4-dcec-4954-a712-5c2d7fdb77d4.


In [80]:
terra.waitForSubmission(refwm, [submission_id1,submission_id2])

1.0 of jobs Succeeded in submission 0.sion 0. 9 mn elapsed.
1.0 of jobs Succeeded in submission 1.


[]

In [ ]:
#refwm.delete_sample_set('19Q3add')

# Downloading from terra

In [18]:
res = refwm.get_sample_sets().loc[samplesetname]
res

aggregateMAF                                                                      NaN
filtered_CGA_MAF_aggregated         gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
passedCGA_filteredMAF_aggregated                                                  NaN
samples                             [ibm_ACH-002446, ibm_ACH-000658, ibm_ACH-00180...
unfiltered_CGA_MAF_aggregated       gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
Name: 19Q3, dtype: object

In [19]:
res = res['filtered_CGA_MAF_aggregated']
! gsutil cp $res "temp/mutation.mergedmaf.txt"



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/275c53c4-dcec-4954-a712-5c2d7fdb77d4/aggregateMAFs_selectFields_workflow/1be5b76f-cabb-49e0-9a4a-c51e2a1a93d3/call-aggregateMAFs_selectFields/19Q3.mergedMAF.txt...
- [1 files][  6.1 MiB/  6.1 MiB]                                                
Operation completed over 1 objects/6.1 MiB.                                      


# postprocessing

In [5]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
load('../JKBio/gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)

2019-07-29 09:18:43::WARNING  R[write to console]: 
Attachement du package : ‘dplyr’


2019-07-29 09:18:43::WARNING  R[write to console]: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


2019-07-29 09:18:43::WARNING  R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


2019-07-29 09:18:43::WARNING  R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


2019-07-29 09:18:44::WARNING  R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

2019-07-29 09:18:44::WARNING  R[write to console]: ✔ ggplot2 3.2.0     ✔ readr   1.3.1
✔ tibble  2.1.3     ✔ purrr   0.3.2
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ ggplot2 3.2.0     ✔ forcats 0.4.0

2019-07-29 09:18:44::WARNING  R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_confl

2019-07-29 09:18:47::WARNING  R[write to console]: 
Attachement du package : ‘S4Vectors’


2019-07-29 09:18:47::WARNING  R[write to console]: The following objects are masked from ‘package:data.table’:

    first, second


2019-07-29 09:18:47::WARNING  R[write to console]: The following object is masked from ‘package:plotly’:

    rename


2019-07-29 09:18:47::WARNING  R[write to console]: The following object is masked from ‘package:tidyr’:

    expand


2019-07-29 09:18:47::WARNING  R[write to console]: The following objects are masked from ‘package:dplyr’:

    first, rename


2019-07-29 09:18:47::WARNING  R[write to console]: The following object is masked from ‘package:plyr’:

    rename


2019-07-29 09:18:47::WARNING  R[write to console]: The following object is masked from ‘package:base’:

    expand.grid


2019-07-29 09:18:47::WARNING  R[write to console]: Le chargement a nécessité le package : IRanges

2019-07-29 09:18:49::WARNING  R[write to console]: 
Attachement du package 

In [6]:
%%R
source('CCLE_postp_function.R')
release <- '19Q3'

2019-07-29 09:18:54::WARNING  R[write to console]: 



In [7]:
%%R
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed
newly_merged_maf <- 'temp/mutation.mergedmaf.txt'

In [8]:
%%R
newly_merged_maf <- readMutations(newly_merged_maf)

In [9]:
%%R
new_release <- createSNPs(newly_merged_maf)

In [10]:
%%R
names(new_release)

 [1] "Hugo_Symbol"            "Entrez_Gene_Id"         "Center"                
 [4] "NCBI_Build"             "Chromosome"             "Start_position"        
 [7] "End_position"           "Strand"                 "Variant_Classification"
[10] "Variant_Type"           "Reference_Allele"       "Tumor_Seq_Allele1"     
[13] "Tumor_Seq_Allele2"      "dbSNP_RS"               "dbSNP_Val_Status"      
[16] "Genome_Change"          "Annotation_Transcript"  "Tumor_Sample_Barcode"  
[19] "cDNA_Change"            "Codon_Change"           "Protein_Change"        
[22] "t_alt_count"            "t_ref_count"            "tumor_f"               
[25] "isDeleterious"          "isTCGAhotspot"          "TCGAhsCnt"             
[28] "isCOSMIChotspot"        "COSMIChsCnt"            "ExAC_AF"               
[31] "PASS"                   "pass"                  


### (1.5) Quick adding

Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

In [11]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe',  data.version=3, data.file='mutations.19Q2')
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == '') {
 previous.release.maf[,1] <- NULL 
}

Fetching https://cds.team/taiga/api/dataset/depmap-mutations-maf-35fe/3 
Status 200 


2019-07-29 09:21:43::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.



loading cached data version from  /Users/jeremie/.taiga/depmap-mutations-maf-35fe_3.toc 


2019-07-29 09:21:43::WARNING  R[write to console]: Loading from cached file /Users/jeremie/.taiga/c7118ff2aa0242379421eb597d60a01d_mutations-19q2.rds



In [57]:
%%R
names(previous.release.maf)

 [1] "Hugo_Symbol"            "Entrez_Gene_Id"         "NCBI_Build"            
 [4] "Chromosome"             "Start_position"         "End_position"          
 [7] "Strand"                 "Variant_Classification" "Variant_Type"          
[10] "Reference_Allele"       "Tumor_Seq_Allele1"      "dbSNP_RS"              
[13] "dbSNP_Val_Status"       "Genome_Change"          "Annotation_Transcript" 
[16] "Tumor_Sample_Barcode"   "cDNA_Change"            "Codon_Change"          
[19] "Protein_Change"         "isDeleterious"          "isTCGAhotspot"         
[22] "TCGAhsCnt"              "isCOSMIChotspot"        "COSMIChsCnt"           
[25] "ExAC_AF"                "CGA_WES_AC"             "SangerWES_AC"          
[28] "SangerRecalibWES_AC"    "RNAseq_AC"              "HC_AC"                 
[31] "RD_AC"                  "WGS_AC"                


In [38]:
%%R
merged <- addToMainMutation(previous.release.maf, new_release)

In [26]:
%%R
names(merged)

 [1] "Tumor_Sample_Barcode"   "Chromosome"             "Start_position"        
 [4] "End_position"           "Reference_Allele"       "Tumor_Seq_Allele1"     
 [7] "ExAC_AF"                "isDeleterious"          "isCOSMIChotspot"       
[10] "COSMIChsCnt"            "isTCGAhotspot"          "TCGAhsCnt"             
[13] "CGA_WES_AC"             "Hugo_Symbol"            "Entrez_Gene_Id"        
[16] "NCBI_Build"             "Strand"                 "Variant_Classification"
[19] "Variant_Type"           "dbSNP_RS"               "dbSNP_Val_Status"      
[22] "Genome_Change"          "Annotation_Transcript"  "cDNA_Change"           
[25] "Codon_Change"           "Protein_Change"         "SangerWES_AC"          
[28] "SangerRecalibWES_AC"    "RNAseq_AC"              "HC_AC"                 
[31] "RD_AC"                  "WGS_AC"                


In [46]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

[1] 0


In [33]:
%%R
dim(merged)

[1] 1277476      32


In [41]:
%%R
filtered <- filterAllelicFraction(merged)

[1] "hey"
[1] 1277476      40


In [42]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [43]:
%%R
merged %<>% dplyr::rename(Tumor_Seq_Allele2=Tumor_Seq_Allele1)

In [44]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

[1] "find conflicts..."
[1] "2.02390491565069 for Hugo_Symbol"
[1] "2.01842192014058 for Entrez_Gene_Id"
[1] "1.74657838344574 for Variant_Classification"
[1] "1.77387266556422 for Variant_Type"
[1] "1.96239206790924 for dbSNP_RS"
[1] "2.08766891956329 for dbSNP_Val_Status"
[1] "2.56088551680247 for Genome_Change"
[1] "2.33601163228353 for Annotation_Transcript"
[1] "2.23472798268 for cDNA_Change"
[1] "2.14647538264592 for Codon_Change"
[1] "1.68733248313268 for Protein_Change"
[1] "1.5594491481781 for isDeleterious"
[1] "1.50205155213674 for isTCGAhotspot"
[1] "1.58020746707916 for TCGAhsCnt"
[1] "1.78558931748072 for isCOSMIChotspot"
[1] "2.0446158806483 for COSMIChsCnt"
[1] "1.95067628622055 for ExAC_AF"
[1] "combining..."
[1] "resolve conflicts.."


In [45]:
%%R
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))

In [47]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)

# Validation

#### Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [13]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')

FileNotFoundError: [Errno 2] File b'temp/mutations.19Q3add.tsv' does not exist: b'temp/mutations.19Q3add.tsv'

## Remove less used 

In [34]:
len(mutations.columns)

32

In [35]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831].values

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


array([['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000047',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '161:0', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000954',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '116:116', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000993',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '7:3', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 12

In [49]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [52]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [67]:
ac_data.shape[0]

1250416

In [81]:
allnan

0

In [79]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(100*pos/ac_data.shape[0],end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.97303617356143886

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1.4827065552584125

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



62.178267072718246

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### basic counts

In [8]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,NaN,34:221,34:213,NaN,NaN,NaN,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,NaN,9:0,53:193,NaN,NaN,NaN,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,NaN,118:0,118:0,10:0,NaN,NaN,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,NaN,NaN,NaN,6:28,NaN,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,NaN,27:61,28:62,NaN,NaN,NaN,NaN
5,POU3F1,5453,37,1,38512139,38512139,+,Missense_Mutation,SNP,C,...,False,0.0,NaN,NaN,NaN,5:0,NaN,NaN,NaN,10:4
6,MAST2,23139,37,1,46498028,46498028,+,Silent,SNP,C,...,False,0.0,0.000008,NaN,NaN,NaN,23:63,33:70,NaN,8:22
7,GBP4,115361,37,1,89657103,89657103,+,Silent,SNP,G,...,False,0.0,0.000016,NaN,97:0,96:0,NaN,NaN,NaN,27:15
8,VAV3,10451,37,1,108247170,108247170,+,Splice_Site,SNP,A,...,False,0.0,NaN,NaN,11:32,11:30,NaN,NaN,NaN,15:18
9,NBPF20,100288142,37,1,148346689,148346689,+,Missense_Mutation,SNP,T,...,False,0.0,0.000017,NaN,257:206,241:196,6:34,NaN,NaN,21:42


In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)


QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

# uploading on taiga

In [162]:
! ls temp

9d492e0a-221a-46ab-855b-1b46916d4c0e_CGA_Production_Analysis_Workflow_74827008-4f79-4151-a895-a8d8623526e5_call-mutation_validator_ibm_ACH-001518.validated.maf
DM19Q2_PATHS_CORRECTED_V2.rsem_genes_expected_count.txt.gz
DM19Q2_PATHS_CORRECTED_V2.rsem_genes_tpm.txt.gz
DM19Q2_PATHS_CORRECTED_V2.rsem_transcripts_tpm.txt.gz
b3bcbdac-9567-4285-8309-0b80aa69b270_CGA_Production_Analysis_Workflow_e4e90f92-6418-4813-839a-1eb2bf664dfe_call-mutation_validator_ibm_ACH-001518.validated.maf
cnv_ccle.called.seg
expression.19Q3.counts.tsv
expression.19Q3.tpm.tsv
expression.19Q3.transcripts.tsv
expression.expectedcount.txt.gz_.gstmp
expression.fusion.tsv
expression.genes.tpm.txt.gz_.gstmp
expression.transcript.tpm.txt.gz
fusions.19Q3.filtered.tsv
fusions.19Q3.unfiltered.tsv
ggplot.png
ibm_ACH-000658.hg38.modeled.png
ibm_ACH-001279.hg38.modeled.png
ibm_ACH-001347.rsem.genes.results
ibm_ACH-001347.rsem.isoforms.results
ibm_ACH-001347.star-fusion.fusion_predictions.abridged.tsv
ibm_ACH-001442.rsem.genes.re

In [48]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'},
                 dataset_description="# Updating to "+release+ """
                 
                 reparing the missing column problem
                 
                 """)

Uploading mutations.19Q3...
hitting https://cds.team/taiga/api/datafile/e0dc6e800c4c45398aa5cd83ce9935d1
Conversion and upload...:
	 Uploading to S3

	 Done: mutations.19Q3 properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 11275e0c82994cd5b6d13be0cd6a14c0 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/11275e0c82994cd5b6d13be0cd6a14c0


'11275e0c82994cd5b6d13be0cd6a14c0'

## comparing two pipelines on same sampleset